<a href="https://colab.research.google.com/github/Rayryu/Should-I-read-this-hespress-article/blob/master/Should_I_read_this_Hespress_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping data

## Robot.txt

Hespress's robot.txt file allows crawling any url ending with .html with is perfect for this project since we are only extraction articles data. 


```
User-agent: Alexabot
Disallow:

User-agent: *
Disallow: /*?print$
Disallow: /*/abuse.php
Disallow: /abuse.php
Disallow: /mobile/abuse.php
Disallow: /*/mobile/abuse.php
```



## Setting up environment and imports

In [0]:
!pip install selenium

In [0]:
import sys
from selenium import webdriver
import pandas as pd

## Running a chromdriver

We're setting up a chromiumdriver in colab.

In [0]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Here Selenium opens the chrome driver wihtout opening a new window.

In [0]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome("chromedriver", options=chrome_options)

## Extracting data 

The get() method will make the driver navigate to the given URL, and **wait** until the page is fully loaded! (unlike request library)

In [0]:
URL = "https://www.hespress.com/societe/469822.html"

driver.get(URL)

The HTML page is fully loaded now, and we can access it's content using driver's methods.

Article's title

In [0]:
title_element = driver.find_elements_by_xpath("//h1[@class='page_title']")  
title = title_element[0].text

In [238]:
print(title)

وزارة التربية تنفي "قرار السبت" وتطالب باستئناف الدراسة عن بُعد


Article's body

In [0]:
body_element = driver.find_element_by_xpath("//div[@id='article_body']") 
body = body_element.text

In [240]:
print(body)

هسبريس - نورالدين إكجان
السبت 02 ماي 2020 - 14:00
منهكة من تتبع تناسل صفحات وأخبار زائفة، جددت وزارة التربية الوطنية نفيها لحسابات إلكترونية باسم سعيد أمزازي، تشهد تعاليق كثيفة من تلاميذ وطلبة، يتفاعل معها صاحب الحساب بشكل غير لائق أحيانا، ما حتم الخروج لنفي صلة الوزير بها.
وعلى امتداد الأسبوع الجاري، تناقلت العديد من الصفحات إمكانية الإفراج عن خبر استثنائي اليوم السبت، يتعلق أساسا بمستقبل السنة الدراسية، وهو ما تنفيه الوزارة جملة وتفصيلا، مطالبة المتعلمين بالانكباب على التحصيل الدراسي.
ومنذ بداية فترة "الحجر الصحي"، خرجت وزارة التربية الوطنية لنفي صحة أخبار تتعلق بقرار سنة بيضاء أو إنجاح جميع التلاميذ أو توقيف الدراسة إلى غاية إشعار آخر، وهو ما أربك عمل الوزارة على امتداد فترة اعتماد "التعليم عن بعد".
وإلى جانب الحسابات والصفحات المزورة، اشتكت الوزارة من اختراق الموقع الرسمي لقطاع التعليم العالي، وهو ما دفعها إلى الاستعانة بالسلطات المختصة من أجل التحري بخصوص الموضوع، وجر المسؤولين عن ذلك إلى القضاء.
وحسب مصادر مسؤولة من داخل وزارة التربية الوطنية، فلا وجود نهائيا لقرار سيتخذ اليوم ال

we need to remove the first wo lines (author and date)

In [231]:
body_splitted = body.split("\n")
author = body_splitted[0].split("-")[1]
date = body_splitted[1]
body = "".join(body.split("\n")[2:])

IndexError: ignored

In [90]:
print(author)

 نورالدين إكجان


In [91]:
print(date)

السبت 02 ماي 2020 - 14:00


Comments and votes

In [0]:
comments_list_element = driver.find_elements_by_class_name("comment_text") 

In [0]:
comments = [comment.text for comment in comments_list_element]

In [0]:
votes_list_element = driver.find_elements_by_class_name("result") 

In [0]:
votes = [vote.text for vote in votes_list_element]

## Building a dataframe

In [0]:
data = pd.DataFrame(data={"comment":comments, "vote": votes})
data["author"] = author
data["title"] = title
data["text"] = body
data["date"] = date
data = data[['author', 'title', 'text', 'comment', 'vote','date']]

In [125]:
data.head()

,author,title,text,comment,vote,date
0,نورالدين إكجان,"وزارة التربية تنفي ""قرار السبت"" وتطالب باستئنا...",منهكة من تتبع تناسل صفحات وأخبار زائفة، جددت و...,تاحنا ماشي شغلنا ماعمرهوم مايديرو شي حاجة كاتف...,504,السبت 02 ماي 2020 - 14:00
1,نورالدين إكجان,"وزارة التربية تنفي ""قرار السبت"" وتطالب باستئنا...",منهكة من تتبع تناسل صفحات وأخبار زائفة، جددت و...,الله يرحم الوالدين اعطيونا شي خبر يقين باش نعر...,912,السبت 02 ماي 2020 - 14:00
2,نورالدين إكجان,"وزارة التربية تنفي ""قرار السبت"" وتطالب باستئنا...",منهكة من تتبع تناسل صفحات وأخبار زائفة، جددت و...,كتلميذ مغربي لا يمكننا مواصلة هكذا دون معرفة م...,429,السبت 02 ماي 2020 - 14:00
3,نورالدين إكجان,"وزارة التربية تنفي ""قرار السبت"" وتطالب باستئنا...",منهكة من تتبع تناسل صفحات وأخبار زائفة، جددت و...,الدراسة عن بعد لاحول ولا قوة الابالله. الناس م...,537,السبت 02 ماي 2020 - 14:00
4,نورالدين إكجان,"وزارة التربية تنفي ""قرار السبت"" وتطالب باستئنا...",منهكة من تتبع تناسل صفحات وأخبار زائفة، جددت و...,صفحة الوزير لي فيها التعليقات راها موثقة ببطاق...,185,السبت 02 ماي 2020 - 14:00


We can extract more features from the HTML page (like comment's date, article image...) but we will restrict ourselves to the 6 attributes that we have chosen.

## Wrapping up everything


We want to write a function that takes in parameters an article's URL and returns a Pandas DataFrame with its content.

In [0]:
import sys
from selenium import webdriver
import pandas as pd
import time

def extract_data(url: str) -> pd.DataFrame:
    """Navigate to the giving URL and extract data from it's content.

    Parameters:
    url (string): article's url.

    Returns:
    pd.DataFrame: a Panda's DataFrame containing article's data.

    """
    # Setting up environnement and creating a driver object
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome("chromedriver", options=chrome_options)
    time.sleep(1)

    # Navigate to the given URL, and wait until the page is fully loaded!
    driver.get(url)
    time.sleep(5)

    # Extracting fields
    title_element = driver.find_elements_by_xpath("//h1[@class='page_title']")  
    title = title_element[0].text

    time.sleep(2)
    body_element = driver.find_elements_by_id("article_body") 
    body = body_element[0].text
    body_splitted = body.split("\n")
    author = body_splitted[0].split("-")[1] if "-" in body_splitted[0] else body_splitted[0]
    date = body_splitted[1]
    body = "".join(body.split("\n")[2:])

    
    comments_list_element = driver.find_elements_by_class_name("comment_text") 
    comments = [comment.text for comment in comments_list_element]
    
    votes_list_element = driver.find_elements_by_class_name("result") 
    votes = [vote.text for vote in votes_list_element]

    # Building a Pandas Dataframe 
    data = pd.DataFrame(data={"comment":comments, "vote": votes})
    data["author"] = author
    data["title"] = title
    data["text"] = body
    data["date"] = date
    data = data[['author', 'title', 'text', 'comment', 'vote','date']]

    return data

In [251]:
URL = "https://www.hespress.com/societe/469866.html"
data = extract_data(URL)
data.head()

,author,title,text,comment,vote,date
0,هسبريس من الرباط,جمعية توضح سياق صرف منح لموظفين بالسجن,"ردا على ما جرى تداوله، مؤخرا، بخصوص ""صرف منحة ...",شكرا للجمعية على المبادرة على كل حال. اما مندو...,8,السبت 02 ماي 2020 - 19:35
1,هسبريس من الرباط,جمعية توضح سياق صرف منح لموظفين بالسجن,"ردا على ما جرى تداوله، مؤخرا، بخصوص ""صرف منحة ...",أتعجب لما إدارة من المؤسسات المغربية او جمعية ...,3,السبت 02 ماي 2020 - 19:35
2,هسبريس من الرباط,جمعية توضح سياق صرف منح لموظفين بالسجن,"ردا على ما جرى تداوله، مؤخرا، بخصوص ""صرف منحة ...",نتق في جمعية التكافل وننوه بكل ما تقوم به في س...,5,السبت 02 ماي 2020 - 19:35
3,هسبريس من الرباط,جمعية توضح سياق صرف منح لموظفين بالسجن,"ردا على ما جرى تداوله، مؤخرا، بخصوص ""صرف منحة ...",اعلان فالمغرب ماكاين اش خدمة مضبوطة(اكاديمية)....,1,السبت 02 ماي 2020 - 19:35
4,هسبريس من الرباط,جمعية توضح سياق صرف منح لموظفين بالسجن,"ردا على ما جرى تداوله، مؤخرا، بخصوص ""صرف منحة ...",الطابع شبه العسكري للقطاع والاعباء الجسيمة الم...,8,السبت 02 ماي 2020 - 19:35
